## Модель и квантование

* Base model: Qwen/Qwen2.5-7B-Instruct
* Загрузка: 4-bit через bitsandbytes
  * nf4
  * double quant = True
  * compute dtype: float16
  * Формат обучения: SFT (supervised fine-tuning) по messages → apply_chat_template → поле `text`

---

## LoRA-конфиг

* r = 16
* lora_alpha = 32
* lora_dropout = 0.05
* bias = "none"
* target_modules = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]

---

## Тренировка

* num_train_epochs = 1
* per_device_train_batch_size = 1
* gradient_accumulation_steps = 8  
  → эффективный batch ≈ 8 (по шагам)
* max_seq_length = 2048
* packing = True  
  (важно: уплотняем несколько коротких примеров в один блок до 512)
* learning_rate = 1e-4
* scheduler: cosine
* warmup: 0.03
* optimizer: paged_adamw_8bit
* fp16 = True
* gradient_checkpointing = True
* max_grad_norm = 1.0
* логи: logging_steps = 20
* eval: eval_steps = 100
* save: save_steps = 100
* (если добавлял) neftune_noise_alpha = 5 — регуляризация против лупов

##  ШАГ 1: ЛЕЧЕНИЕ ОКРУЖЕНИЯ + УСТАНОВКА

In [1]:
# 1. Сначала лечим фундамент
!pip install -q -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q transformers==4.46.0 trl==0.8.6 peft bitsandbytes accelerate datasets

import torch
print(f"Torch version: {torch.__version__}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.6 MB/s eta 0:00:00
Reason for being yanked: This version unfortunately does not work with 3.8 but we did not drop the support yet
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.3/566.3 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 16.2 MB/s eta 0:00:00
Torch version: 2.9.0+cu126 (Ошибки импорта больше нет)


In [2]:
# загрузка файлов
from google.colab import files

uploaded = files.upload()

Saving train.jsonl to train.jsonl
Saving valid.jsonl to valid.jsonl


In [3]:
!ls -lh

total 2.1M
drwxr-xr-x 1 root root 4.0K Dec  9 14:42 sample_data
-rw-r--r-- 1 root root 2.0M Feb  3 15:13 train.jsonl
-rw-r--r-- 1 root root 104K Feb  3 15:13 valid.jsonl


In [4]:
!mkdir -p data
!mv train.jsonl valid.jsonl data/
!ls data

train.jsonl  valid.jsonl


In [9]:
# чекап
!nvidia-smi

Tue Feb  3 15:42:57 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   71C    P0             32W /   70W |   14192MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
!ls -lh
!head -n 1 data/train.jsonl
!head -n 1 data/valid.jsonl

total 8.0K
drwxr-xr-x 2 root root 4.0K Feb  3 15:13 data
drwxr-xr-x 1 root root 4.0K Dec  9 14:42 sample_data
{"messages": [{"role": "system", "content": "Ты — поэтический генератор песен в стиле группы «Макулатура». Соблюдай структуру и теги секций (<VERSE>, <CHORUS>, <BRIDGE>, <INTRO>, <OUTRO>, <REFRAIN>, <HOOK>). Сохраняй голос, заданный атрибутом speaker (alekhin/speransky/group). Пиши по-русски. Не добавляй пояснений, только текст песен/секций в требуемом формате."}, {"role": "user", "content": "Продолжи песню в стиле группы «Макулатура».\nТема: поездка и расставание\nНиже приведён контекст (последние секции). \nНапиши СЛЕДУЮЩУЮ секцию. Обязательное требование: голос = speransky.\nВерни ровно один блок секции с тегами.\n\nКОНТЕКСТ:\n<VERSE index=1 speaker=alekhin>\nБессонница, Гомер\nи\nНед Фландерс\nРешая простейший вопрос, поругались\nМне было неплохо, когда мы расстались\nНо в руках все ломалось, во мне много страсти\nИ я думал, мы будем вместе до старости\nИ я утешал себя, «вс

## ШАГ 2: ЗАГРУЗКА (Адаптировано под T4)

In [7]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, prepare_model_for_kbit_training

MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"

# 1. Датасет
data_files = {"train": "data/train.jsonl", "validation": "data/valid.jsonl"}
ds = load_dataset("json", data_files=data_files)

# 2. Токенизатор
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

def to_text(example):
    return {"text": tokenizer.apply_chat_template(example["messages"], tokenize=False, add_generation_prompt=False)}

ds = ds.map(to_text, remove_columns=ds["train"].column_names)

# 3. Модель (С защитой от BFloat16)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,  # <--- float16
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

print(f"Загружаем {MODEL_NAME}...")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,             # <--- float16
    trust_remote_code=True
)

model.config.use_cache = False
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16, lora_alpha=32, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]
)

print("✅ Шаг 2 пройден. Ошибок импорта нет. Можно запускать Тренер!")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

✅ Шаг 2 пройден. Ошибок импорта нет. Можно запускать Тренер!


## ШАГ 3: Тренировка

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments

OUTPUT_DIR = "qwen_makulatura_lora"

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,

    evaluation_strategy="steps",
    eval_steps=50,
    save_steps=50,
    logging_steps=10,

    learning_rate=1e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,

    optim="paged_adamw_8bit",

    fp16=True,       # Включаем обычный FP16
    bf16=False,      # Строго выключаем BF16 (иначе T4 умрет)

    gradient_checkpointing=True,
    max_grad_norm=1.0,
    report_to="none",
    neftune_noise_alpha=5,
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,             # Работает в версии 0.8.6!
    train_dataset=ds["train"],
    eval_dataset=ds["validation"],
    peft_config=lora_config,
    dataset_text_field="text",       # Работает в версии 0.8.6!

    max_seq_length=2048,             # Увеличил, чтобы песни влезали
    packing=True,

    dataset_kwargs={"add_special_tokens": False},
    args=training_args,
)

print("🚀 Запуск по старым чертежам...")
trainer.train()

/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:323: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(


🚀 Запуск по старым чертежам...


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

## 4) Сохранить адаптер (LoRA)

Это уже готовый результат: папка qwen_makulatura_lora/

In [ ]:
import os

OUTPUT_DIR = "qwen_makulatura_lora"

# Сохраняем веса адаптера
trainer.model.save_pretrained(OUTPUT_DIR)
# Сохраняем токенизатор (важно для чат-шаблонов)
tokenizer.save_pretrained(OUTPUT_DIR)

print("Адаптеры сохранены")
!ls -lah qwen_makulatura_lora

In [ ]:
# ОЧИСКА ПАМЯТИ (Обязательно)
import torch
import gc

# Удаляем объекты обучения
try:
    del model
    del trainer
except NameError:
    pass

# Чистим кэш CUDA
gc.collect()
torch.cuda.empty_cache()

print("🧹 Память видеокарты очищена. Можно загружать модель для генерации.")

🧹 Память видеокарты очищена. Можно загружать модель для генерации.


## 5) Генерация

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

# --- 1. ПЕРЕЗАГРУЗКА ---
if 'model' not in locals():
    MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"
    OUTPUT_DIR = "qwen_makulatura_lora"

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
    )

    base_model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME, quantization_config=bnb_config, device_map="auto", torch_dtype=torch.float16, trust_remote_code=True
    )

    model = PeftModel.from_pretrained(base_model, OUTPUT_DIR)
    tok = AutoTokenizer.from_pretrained(OUTPUT_DIR)
# -----------------------------------------------

# внутренний промпт
system_prompt = """Ты — нейросеть, генерирующая тексты группы «Макулатура».

Жанр: абстрактный хип-хоп / spoken word.
Тон: холодный, отстранённый, интеллектуальный, без аффектации.

Ключевые темы:
— одиночество и отчуждение
— усталость от культуры и языка
— бессмысленные перемещения (поезда, города, отели)
— литература, тексты, письмо
— время, смерть, пустота
— холод, расстояние, девушка как образ, а не персонаж

Формальные требования:
— строго используй теги <VERSE> и <CHORUS>
— каждый тег ОБЯЗАТЕЛЬНО содержит атрибут speaker
— никаких других тегов или форматов
— текст — только песня, без пояснений

Герои и голоса:

[alekhin]
— уставший писатель, скептик
— речь приземлённая, сухая, ироничная
— образы: книги, черновики, купе поездов, гостиницы, пустые города
— дистанция к эмоциям, отказ от пафоса

[speransky]
— метафизик, философ
— речь абстрактная, плотная, метафорическая
— образы: время, смерть, космос, пустота, распад субъекта
— избегает бытовых деталей

Строгие запреты:
— бытовая грязь ради шока (пельмени, водка, бомжи, подъезды)
— прямой социальный репортаж
— романтизация нищеты
— юмор, ирония, мемы
— клишированные рифмы и рэп-штампы

Цель:
создать текст, который мог бы органично существовать в дискографии группы «Макулатура».

# Тема: Интеллектуальная тоска

"""

user_prompt = """Напиши текст песни.
Тема: ночная поездка в такси, город как декорация, чувство, что ты персонаж плохой книги.
Структура:
<VERSE index=1 speaker=alekhin> (монолог про усталость и город)
<CHORUS speaker=group> (абстрактный рефрен)
<VERSE index=2 speaker=speransky> (сюрреализм, время, смерть)
<OUTRO speaker=group>
"""
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

text = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tok([text], return_tensors="pt").to(model.device)

print("📝 Генерация (Format: Canonical)...")

with torch.no_grad():
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=1200,      # Ставим побольше, так как структура сложная
        do_sample=True,
        temperature=0.75,         # Оптимально для сохранения стиля
        top_p=0.9,
        top_k=50,
        repetition_penalty=1.12,
        pad_token_id=tok.eos_token_id
    )

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tok.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("="*40)
print(response)
print("="*40)

📝 Генерация (Format: Canonical)...
<VERSE index=1 speaker=alekhin>
Если бы я мог выбраться на улицу
Наслаждаться звуками ночного города
Поглотить весь этот шум в себя
Забыть о том, что было здесь совсем недавно
Сквозь трещины в полу проползает свет
На стенах отражаются бархатные занавески
Иногда хочется спуститься к своим друзьям
Дать им денег на бутылку водки
Чтобы они напились до полусмерти
А потом отправили меня домой, чтобы я не мешался
Уже не вспоминаю, когда была последняя ночь
Когда был хотя бы немного трезвым
Жизнь всегда возвращает нас к тому же месту
Это и есть жизнь
</VERSE>

<CHORUS speaker=group>
Хочу, чтобы кто-нибудь меня освежил
Открывал холодильник
Не нашел там ничего, кроме пирожков
Выходила из комнаты
Как в лужах заливают старые кошки
Хочу, чтобы кто-нибудь меня освежил
Но уже нет такого времени
И ведь ты такая красивая
Ведь ты такая красивая
</CHORUS>

<VERSE index=2 speaker=speransky>
Я не могу сказать тебе всего этого
Вот почему я пришел сюда один
За чаем я смотре

## 6) Merge LoRA в fp16 модель

Это удобно, если потом конвертировать в MLX или просто запускать одним каталогом.

In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM

MERGED_DIR = "qwen_makulatura_merged_fp16"

# Грузим базу уже в fp16 (без 4bit)
base_fp16 = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)

peft_model = PeftModel.from_pretrained(base_fp16, OUTPUT_DIR)
merged = peft_model.merge_and_unload()

merged.save_pretrained(MERGED_DIR, safe_serialization=True)
tokenizer.save_pretrained(MERGED_DIR)

!ls -lah qwen_makulatura_merged_fp16

In [ ]:
# сейв
from google.colab import files
files.download("qwen_makulatura_lora.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>